In [1]:
import math
import time
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import matplotlib.animation as animation

import MBstructs as MB
import MultiBodyFuncts as MBF
import DCA

np.set_printoptions(precision=4,suppress=True)

#### Kinematic Sweep

In [2]:
# Kinematic sweep is trivial for planar n-link pendulum 
# def kinematics(bodies,q,u):
#     for body,theta,omega in zip(bodies,np.cumsum(q),np.cumsum(u)):
#         print(theta)
#         body.omega = omega
#         print(omega)
#         body.CBN = MBF.DCM(theta)
#         print(body.CBN)
def kinematics(bodies,state):
    
    # slice state into 'qs' for each element 
    qe = state[:len(state0)/2:3]
    theta = np.cumsum(qe)
    
    # slice into the two rotational coordinates for each body
    theta1 = theta[0::2]
    theta2 = theta[1::2]
    for body,q0,q3 in zip(bodies,theta1, theta2):
        body.theta1 = q0
        body.theta2 = q3

#### Used for Plotting

In [3]:
def get_system_topology(state):
    ntsteps,nbodies = np.shape(state)
    q = state[:,:nbodies/2]
    theta = np.cumsum(q,1)
    C = [[np.array([[np.cos(angle),-np.sin(angle)],\
                    [np.sin(angle), np.cos(angle)]]) 
      for angle in tstep] for tstep in theta]
    R = [[np.dot(CBN,np.array([l,0])) for CBN in timestep] \
                                      for timestep in C]
    R = [np.cumsum(r,0) for r in R]
    
    x = [np.hstack((np.array([0]),r[:,0])) for r in R]
    y = [np.hstack((np.array([0]),r[:,1])) for r in R]
    
    return x,y

def sys_update(x, y, line):
    line.set_data(x, y)
    return line,

#### Energy Function for Solution Verification

In [4]:
def get_energy(bodies,state):
    
    ntsteps,nbodies = np.shape(state)
    q = state[:,:nbodies/2]
    u = state[:,nbodies/2:]

    theta = np.cumsum(q,1)
    omega = np.cumsum(u,1)

    C = [[np.array([[np.cos(angle),-np.sin(angle)],\
                    [np.sin(angle), np.cos(angle)]]) for angle in tstep] 
                                                     for tstep in theta]

    ## Potential Energy Calculation ##
    # vector location of joints for all bodies
    Rh2Body = [[np.dot(CBN,np.array([l,0])) 
                                      for CBN in timestep] 
                                      for timestep in C]
    Rh2 = [np.cumsum(r,0) for r in Rh2Body]

    # vector location of center of mass for all bodies
    # from handle two
    rcmBody = [[np.dot(CBN,np.array([-l/2,0])) 
                                        for CBN in timestep]
                                        for timestep in C]

    # locate the centers of mass w.r.t. origin
    rcm = [np.array(Rbody + rbody) for Rbody,rbody in zip(Rh2,rcmBody)]

    # slice out c.m. y position
    ycm = [y[:,1] for y in rcm]

    # compute heights of centers of mass
    hcmRel = [body.l/2 for body in bodies]
    lcm = np.cumsum([body.l for body in bodies])
    lcm = lcm.tolist()
    lcm.pop(len(bodies)-1)
    lcm.insert(0,0)
    lcm = np.array(lcm)
    hcm = [lcm+hcmRel+y for y in ycm]

    pe = np.sum([[9.81*body.m*h 
                for bodies,h in zip(bodies,timestep)]
                for timestep in hcm],1)

    ## Kinetic Energy Calculation ##
    vh2Body = np.cumsum([[qdot*np.array([-R[1],R[0]]) 
                for qdot,R in zip(timestep,R)] 
                for timestep,R in zip(omega,Rh2Body)],1)

    vcmBody = [[qdot*np.array([-r[1],r[0]]) 
                for qdot,r in zip(tstep,r_tstep)] 
                for tstep,r_tstep in zip(omega,rcmBody)]

    vcm = vcmBody+vh2Body

    keT = np.sum([[1/2*body.m*np.dot(v,v) 
                   for body,v in zip(bodies,timestep)] 
                   for timestep in vcm],1)

    keR = np.sum([[1/2*body.I*qdot**2 
                for body,qdot in zip(bodies,timestep)]
                for timestep in omega],1)
    ke = keT + keR
    te = ke+pe
    return ke,pe,te

#### Function that calls kinematics, updates bodies inertial properties and calls DCA

In [5]:
def simulate(state,tspan,n,bodies,joints,BC1,BC2):
    '''
    his function extracts the generalized coordinates from the solution 
    of the equations of motion after calling the DCA to solve the equations of motion 
    '''
    q = state[:n]
    u = state[n:]
    # Get the Kinematics 2D n-link Pendulum
    kinematics(bodies,state)    
    
    # slice out generalized cooridnates
    # and make a sublist for each body
    u = state[:len(state)/2].tolist()
    u = [u[i*6:(i*6)+6] for i in range((int(len(u)/6)))]

    # 'pop' off the rotational coordinates 
    # those are dealt with through the 'kinematic' sweep
    for ue in u:
        ue.pop(3)
        ue.pop(0)
        
    # compute the inverse inertial properties of the body 
    # with the updated generalized 
    for body,ue in zip(bodies,u):
        body.intProps('gebf',ue)
    
    # DCA returns the eddot vector
    sol = DCA.solve(nbodies,0,bodies,joints,BC1,BC2)
    
    qddot = np.array([qddot for qddot_body in sol for qddot in qddot_body],dtype=np.double).reshape(nGEBF*6)
    state_dot = np.hstack((state[len(state)/2:],qddot))
    return state_dot 
    
    #Call the Recursive DCA Algorithm
    #This returns a list of the form:
    #[A11,A12,A21,A22,...,An1,An2]
    #where Axy corresponds to the acceleration
    #of the yth handle of the xth body
#     ll=DCA.openR(n,0,bodies,joints,BC1,BC2)

#     #loop to fill d_dt with the acceleration values
#     d_dt=np.zeros((2*n))
#     for j in range(0,n):
#         if j == 0:
#             A1=ll.pop(0)
#             d_dt[j+n]=np.dot(np.transpose(joints[j].P),A1)
#         else:
#             A2= ll.pop(0)
#             A1=ll.pop(0)
#             d_dt[j+n]=np.dot(np.transpose(joints[j].P),(A1-A2))
    
#     #add the velocities to d_dt and return to the integrator
#     d_dt[:n]=state[n:]
#     return d_dt 

#### System Initilization

In [6]:
# number of bodies and GEBF elements and kinematic joints
nGEBF = 2
nRIGID = 0
nJKIN = 1
nbodies = nGEBF + nRIGID

# number of degrees of freedom for the 
# 2D-GEBF element with a single planar rotation 
# and two planar displacements (2 nodes per element)
ndofsGEBF = 6
ndofsKIN  = 1 # (one pin joint)

# GEBF-Properties
# Physical Properties
A   =  0.0018
I   =  1.215e-8
L   =  1.2
r   =  math.sqrt(A/math.pi)
l   =  L/nGEBF

# Material Properties
E   =  0.7e6
rho =  5540



# Create a list of Joints
jointsKIN = [MB.Joint('revolute2D') for i in range(nJKIN)]
jointsGEBF = [MB.Joint('gebf') for i in range (nGEBF-1)]
joints = jointsKIN + jointsGEBF

# Create a the list of bodies 
bodiesGEBF = [MB.Body() for i in range(nGEBF)]
for body in bodiesGEBF:
    body.initialize('gebf', E, A, I, r, rho, l)
bodies = bodiesGEBF

# for body in bodiesRIGID:
#     body.initialize(m,l,I)
# bodies = bodiesRIGID

#### Simulation Specifications

In [7]:
# start the rope from the horizontal 
# Compute initial generalized coordinates
# q = [theta1, u_1x, u_1y, theta2, u_2x','u_2y']
# u = q_dot
# state = [q, u]

# sublists handy for initilization of bodies 
q0GEBF = [[0 for i in range(ndofsGEBF)] for i in range(nGEBF)]
u0GEBF = [[0 for i in range(ndofsGEBF)] for i in range(nGEBF)]

# flatten sub-lists into a single state vector
q0 = np.array([q for q_body in q0GEBF for q in q_body],dtype=np.double).squeeze()
u0 = np.array([u for u_body in u0GEBF for u in u_body],dtype=np.double).squeeze()
state0 = np.hstack((q0,u0))

# Length of time of the simulation
tspan = np.arange(0,.0002,.0001)

#### Simulate it

In [8]:
# odeint is the numerical integrator used
state = odeint(simulate,state0,tspan,(nbodies,bodies,joints,2,1))

In [9]:
# x,y = get_system_topology(state)

In [10]:
# # First set up the figure, the axis, and the plot element we want to animate
# fig = plt.figure()
# ax = plt.axes(xlim=(-3, 3), ylim=(-3, 3))
# line, = ax.plot([], [], lw=2)

# # initialization function: plot the background of each frame
# def init():
#     line.set_data([], [])
#     return line,

# # animation function.  This is called sequentially
# def animate(i):
#     line.set_data(x[i], y[i])
#     return line,

# # call the animator.  blit=True means only re-draw the parts that have changed.
# anim = animation.FuncAnimation(fig, animate, init_func=init,
#                                frames=len(x), repeat=False, interval=1, blit=False)

# # save the animation as an mp4.  This requires ffmpeg or mencoder to be
# # installed.  The extra_args ensure that the x264 codec is used, so that
# # the video can be embedded in html5.  You may need to adjust this for
# # your system: for more information, see
# # http://matplotlib.sourceforge.net/api/animation_api.html
# # anim.save('basic_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])

# plt.show()

In [11]:
# ke,pe,te = get_energy(bodies,state)

In [12]:
# # The energy of the system is calculated and plotted
# %matplotlib inline
# plt.plot(tspan,te-te[0])
# plt.xlabel("Time [s]")
# plt.ylabel("energy")
# plt.title("System Energy")
# plt.show()

# plt.plot(tspan,pe,tspan,ke)
# plt.xlabel("Time[s]")
# plt.ylabel("energy")
# plt.title("Kinetic and Potential Energy")
# plt.show

In [13]:
for y in state:
    print(y)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.]
[-0.      0.      0.      0.0002 -0.      0.      0.0002 -0.      0.
  0.0001 -0.      0.     -0.7071  0.      0.      3.1512 -0.0659  0.0661
  3.1512 -0.0659  0.0661  1.0759 -0.0304  0.0305]
